## **核函数**
作用：  
将数据从某个特征空间映射到另一个特征空间  

径向基高斯核函数：
$$
k(x,y)=exp\frac{-\left|\left|x-y\right|\right|^2}{2\sigma^2}
$$

In [1]:
import numpy as np

核函数转换

In [2]:
def kernelTrans(X, A, kTup):
    m, n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = np.exp(K / (-1 * kTup[1]**2))
    else:
        raise NameError('Houston We Have a Problem That Kernel is not recongnized')
    return K

Platt SMO支持函数

In [3]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))    # 误差缓存
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)
            
# 计算E值并返回
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * oS.K[:, k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 选择内循环的alpha值
def selectJ(i, oS, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validecachelist = np.nonzero(oS.eCache[:, 0].A)[0]    # 返回非0的E值对应的alpha值
    if len(validecachelist) > 1:
        for k in validecachelist:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:    # 选择具有最大步长的j
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]
    
# 读取数据，获得特征与标签
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

# i是alpha下标，m是alpha数目
def selectJrand(i, m):
    j = i;
    while j == i:
        j = int(np.random.uniform(0, m))
    return j

# 调整大于H或小于L的alpha值
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

Platt SMO优化例程

In [4]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] >0)):
        j ,Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        # 保证alpha在0与C之间
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] - oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] - oS.alphas[i])
        if L == H:
            print('L == H')
            return 0
        # alpha[j]的最优修改量
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]
        if eta >= 0:
            print('eta >= 0')
            return 0
        # 计算新的alpha[j]
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, i)
        # 设置常数项b
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, i] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, j] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j, j]
        if 0 < oS.alphas[i] and oS.C > oS.alphas[i]:
            oS.b = b1
        elif 0 < oS.alphas[j] and oS.C > oS.alphas[j]:
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

Platt SMO外循环代码

In [5]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while iter < maxIter and (alphaPairsChanged > 0 or entireSet):
        alphaPairsChanged = 0
        if entireSet:    # 遍历所有值
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print('fullSet, iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter += 1
        else:    # 遍历非边界值
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print('non-bound, iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        print('iteration number: %d' % iter)
    return oS.b, oS.alphas

计算w

In [6]:
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

测试函数

In [7]:
def testRbf(k1=1.3):
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print('there are %d Support Vectors' % np.shape(sVs)[0])
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print('the training error rate is: %f' % (float(errorCount)/m))
    dataArr, labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print('the test error rate is: %f' % (float(errorCount)/m))

In [8]:
testRbf()

L == H
fullSet, iter: 0 i: 0, pairs changed 0
fullSet, iter: 0 i: 1, pairs changed 1
fullSet, iter: 0 i: 2, pairs changed 2
fullSet, iter: 0 i: 3, pairs changed 3
fullSet, iter: 0 i: 4, pairs changed 3
fullSet, iter: 0 i: 5, pairs changed 4
L == H
fullSet, iter: 0 i: 6, pairs changed 4
fullSet, iter: 0 i: 7, pairs changed 4
fullSet, iter: 0 i: 8, pairs changed 4
fullSet, iter: 0 i: 9, pairs changed 4
fullSet, iter: 0 i: 10, pairs changed 4
fullSet, iter: 0 i: 11, pairs changed 4
fullSet, iter: 0 i: 12, pairs changed 4
fullSet, iter: 0 i: 13, pairs changed 4
fullSet, iter: 0 i: 14, pairs changed 5
fullSet, iter: 0 i: 15, pairs changed 6
fullSet, iter: 0 i: 16, pairs changed 6
fullSet, iter: 0 i: 17, pairs changed 7
fullSet, iter: 0 i: 18, pairs changed 8
fullSet, iter: 0 i: 19, pairs changed 8
fullSet, iter: 0 i: 20, pairs changed 8
fullSet, iter: 0 i: 21, pairs changed 9
fullSet, iter: 0 i: 22, pairs changed 9
fullSet, iter: 0 i: 23, pairs changed 9
fullSet, iter: 0 i: 24, pairs change

non-bound, iter: 18 i: 0, pairs changed 1
non-bound, iter: 18 i: 1, pairs changed 2
non-bound, iter: 18 i: 2, pairs changed 3
non-bound, iter: 18 i: 3, pairs changed 4
non-bound, iter: 18 i: 5, pairs changed 5
non-bound, iter: 18 i: 14, pairs changed 6
iteration number: 19
non-bound, iter: 19 i: 0, pairs changed 1
non-bound, iter: 19 i: 1, pairs changed 2
non-bound, iter: 19 i: 2, pairs changed 3
non-bound, iter: 19 i: 3, pairs changed 4
non-bound, iter: 19 i: 5, pairs changed 5
non-bound, iter: 19 i: 14, pairs changed 6
iteration number: 20
non-bound, iter: 20 i: 0, pairs changed 1
non-bound, iter: 20 i: 1, pairs changed 2
non-bound, iter: 20 i: 2, pairs changed 3
non-bound, iter: 20 i: 3, pairs changed 4
non-bound, iter: 20 i: 5, pairs changed 5
non-bound, iter: 20 i: 14, pairs changed 6
iteration number: 21
non-bound, iter: 21 i: 0, pairs changed 1
non-bound, iter: 21 i: 1, pairs changed 2
non-bound, iter: 21 i: 2, pairs changed 3
non-bound, iter: 21 i: 3, pairs changed 4
non-bound,

non-bound, iter: 63 i: 2, pairs changed 3
non-bound, iter: 63 i: 3, pairs changed 4
non-bound, iter: 63 i: 5, pairs changed 5
non-bound, iter: 63 i: 14, pairs changed 6
iteration number: 64
non-bound, iter: 64 i: 0, pairs changed 1
non-bound, iter: 64 i: 1, pairs changed 2
non-bound, iter: 64 i: 2, pairs changed 3
non-bound, iter: 64 i: 3, pairs changed 4
non-bound, iter: 64 i: 5, pairs changed 5
non-bound, iter: 64 i: 14, pairs changed 6
iteration number: 65
non-bound, iter: 65 i: 0, pairs changed 1
non-bound, iter: 65 i: 1, pairs changed 2
non-bound, iter: 65 i: 2, pairs changed 3
non-bound, iter: 65 i: 3, pairs changed 4
non-bound, iter: 65 i: 5, pairs changed 5
non-bound, iter: 65 i: 14, pairs changed 6
iteration number: 66
non-bound, iter: 66 i: 0, pairs changed 1
non-bound, iter: 66 i: 1, pairs changed 2
non-bound, iter: 66 i: 2, pairs changed 3
non-bound, iter: 66 i: 3, pairs changed 4
non-bound, iter: 66 i: 5, pairs changed 5
non-bound, iter: 66 i: 14, pairs changed 6
iteration

non-bound, iter: 108 i: 0, pairs changed 1
non-bound, iter: 108 i: 1, pairs changed 2
non-bound, iter: 108 i: 2, pairs changed 3
non-bound, iter: 108 i: 3, pairs changed 4
non-bound, iter: 108 i: 5, pairs changed 5
non-bound, iter: 108 i: 14, pairs changed 6
iteration number: 109
non-bound, iter: 109 i: 0, pairs changed 1
non-bound, iter: 109 i: 1, pairs changed 2
non-bound, iter: 109 i: 2, pairs changed 3
non-bound, iter: 109 i: 3, pairs changed 4
non-bound, iter: 109 i: 5, pairs changed 5
non-bound, iter: 109 i: 14, pairs changed 6
iteration number: 110
non-bound, iter: 110 i: 0, pairs changed 1
non-bound, iter: 110 i: 1, pairs changed 2
non-bound, iter: 110 i: 2, pairs changed 3
non-bound, iter: 110 i: 3, pairs changed 4
non-bound, iter: 110 i: 5, pairs changed 5
non-bound, iter: 110 i: 14, pairs changed 6
iteration number: 111
non-bound, iter: 111 i: 0, pairs changed 1
non-bound, iter: 111 i: 1, pairs changed 2
non-bound, iter: 111 i: 2, pairs changed 3
non-bound, iter: 111 i: 3, p

non-bound, iter: 149 i: 3, pairs changed 4
non-bound, iter: 149 i: 5, pairs changed 5
non-bound, iter: 149 i: 14, pairs changed 6
iteration number: 150
non-bound, iter: 150 i: 0, pairs changed 1
non-bound, iter: 150 i: 1, pairs changed 2
non-bound, iter: 150 i: 2, pairs changed 3
non-bound, iter: 150 i: 3, pairs changed 4
non-bound, iter: 150 i: 5, pairs changed 5
non-bound, iter: 150 i: 14, pairs changed 6
iteration number: 151
non-bound, iter: 151 i: 0, pairs changed 1
non-bound, iter: 151 i: 1, pairs changed 2
non-bound, iter: 151 i: 2, pairs changed 3
non-bound, iter: 151 i: 3, pairs changed 4
non-bound, iter: 151 i: 5, pairs changed 5
non-bound, iter: 151 i: 14, pairs changed 6
iteration number: 152
non-bound, iter: 152 i: 0, pairs changed 1
non-bound, iter: 152 i: 1, pairs changed 2
non-bound, iter: 152 i: 2, pairs changed 3
non-bound, iter: 152 i: 3, pairs changed 4
non-bound, iter: 152 i: 5, pairs changed 5
non-bound, iter: 152 i: 14, pairs changed 6
iteration number: 153
non-b

...

non-bound, iter: 9793 i: 0, pairs changed 1
non-bound, iter: 9793 i: 1, pairs changed 2
non-bound, iter: 9793 i: 2, pairs changed 3
non-bound, iter: 9793 i: 3, pairs changed 4
non-bound, iter: 9793 i: 5, pairs changed 5
non-bound, iter: 9793 i: 14, pairs changed 6
iteration number: 9794
non-bound, iter: 9794 i: 0, pairs changed 1
non-bound, iter: 9794 i: 1, pairs changed 2
non-bound, iter: 9794 i: 2, pairs changed 3
non-bound, iter: 9794 i: 3, pairs changed 4
non-bound, iter: 9794 i: 5, pairs changed 5
non-bound, iter: 9794 i: 14, pairs changed 6
iteration number: 9795
non-bound, iter: 9795 i: 0, pairs changed 1
non-bound, iter: 9795 i: 1, pairs changed 2
non-bound, iter: 9795 i: 2, pairs changed 3
non-bound, iter: 9795 i: 3, pairs changed 4
non-bound, iter: 9795 i: 5, pairs changed 5
non-bound, iter: 9795 i: 14, pairs changed 6
iteration number: 9796
non-bound, iter: 9796 i: 0, pairs changed 1
non-bound, iter: 9796 i: 1, pairs changed 2
non-bound, iter: 9796 i: 2, pairs changed 3
non-

non-bound, iter: 9845 i: 3, pairs changed 4
non-bound, iter: 9845 i: 5, pairs changed 5
non-bound, iter: 9845 i: 14, pairs changed 6
iteration number: 9846
non-bound, iter: 9846 i: 0, pairs changed 1
non-bound, iter: 9846 i: 1, pairs changed 2
non-bound, iter: 9846 i: 2, pairs changed 3
non-bound, iter: 9846 i: 3, pairs changed 4
non-bound, iter: 9846 i: 5, pairs changed 5
non-bound, iter: 9846 i: 14, pairs changed 6
iteration number: 9847
non-bound, iter: 9847 i: 0, pairs changed 1
non-bound, iter: 9847 i: 1, pairs changed 2
non-bound, iter: 9847 i: 2, pairs changed 3
non-bound, iter: 9847 i: 3, pairs changed 4
non-bound, iter: 9847 i: 5, pairs changed 5
non-bound, iter: 9847 i: 14, pairs changed 6
iteration number: 9848
non-bound, iter: 9848 i: 0, pairs changed 1
non-bound, iter: 9848 i: 1, pairs changed 2
non-bound, iter: 9848 i: 2, pairs changed 3
non-bound, iter: 9848 i: 3, pairs changed 4
non-bound, iter: 9848 i: 5, pairs changed 5
non-bound, iter: 9848 i: 14, pairs changed 6
ite

iteration number: 9884
non-bound, iter: 9884 i: 0, pairs changed 1
non-bound, iter: 9884 i: 1, pairs changed 2
non-bound, iter: 9884 i: 2, pairs changed 3
non-bound, iter: 9884 i: 3, pairs changed 4
non-bound, iter: 9884 i: 5, pairs changed 5
non-bound, iter: 9884 i: 14, pairs changed 6
iteration number: 9885
non-bound, iter: 9885 i: 0, pairs changed 1
non-bound, iter: 9885 i: 1, pairs changed 2
non-bound, iter: 9885 i: 2, pairs changed 3
non-bound, iter: 9885 i: 3, pairs changed 4
non-bound, iter: 9885 i: 5, pairs changed 5
non-bound, iter: 9885 i: 14, pairs changed 6
iteration number: 9886
non-bound, iter: 9886 i: 0, pairs changed 1
non-bound, iter: 9886 i: 1, pairs changed 2
non-bound, iter: 9886 i: 2, pairs changed 3
non-bound, iter: 9886 i: 3, pairs changed 4
non-bound, iter: 9886 i: 5, pairs changed 5
non-bound, iter: 9886 i: 14, pairs changed 6
iteration number: 9887
non-bound, iter: 9887 i: 0, pairs changed 1
non-bound, iter: 9887 i: 1, pairs changed 2
non-bound, iter: 9887 i: 

non-bound, iter: 9942 i: 14, pairs changed 6
iteration number: 9943
non-bound, iter: 9943 i: 0, pairs changed 1
non-bound, iter: 9943 i: 1, pairs changed 2
non-bound, iter: 9943 i: 2, pairs changed 3
non-bound, iter: 9943 i: 3, pairs changed 4
non-bound, iter: 9943 i: 5, pairs changed 5
non-bound, iter: 9943 i: 14, pairs changed 6
iteration number: 9944
non-bound, iter: 9944 i: 0, pairs changed 1
non-bound, iter: 9944 i: 1, pairs changed 2
non-bound, iter: 9944 i: 2, pairs changed 3
non-bound, iter: 9944 i: 3, pairs changed 4
non-bound, iter: 9944 i: 5, pairs changed 5
non-bound, iter: 9944 i: 14, pairs changed 6
iteration number: 9945
non-bound, iter: 9945 i: 0, pairs changed 1
non-bound, iter: 9945 i: 1, pairs changed 2
non-bound, iter: 9945 i: 2, pairs changed 3
non-bound, iter: 9945 i: 3, pairs changed 4
non-bound, iter: 9945 i: 5, pairs changed 5
non-bound, iter: 9945 i: 14, pairs changed 6
iteration number: 9946
non-bound, iter: 9946 i: 0, pairs changed 1
non-bound, iter: 9946 i:

non-bound, iter: 9995 i: 5, pairs changed 5
non-bound, iter: 9995 i: 14, pairs changed 6
iteration number: 9996
non-bound, iter: 9996 i: 0, pairs changed 1
non-bound, iter: 9996 i: 1, pairs changed 2
non-bound, iter: 9996 i: 2, pairs changed 3
non-bound, iter: 9996 i: 3, pairs changed 4
non-bound, iter: 9996 i: 5, pairs changed 5
non-bound, iter: 9996 i: 14, pairs changed 6
iteration number: 9997
non-bound, iter: 9997 i: 0, pairs changed 1
non-bound, iter: 9997 i: 1, pairs changed 2
non-bound, iter: 9997 i: 2, pairs changed 3
non-bound, iter: 9997 i: 3, pairs changed 4
non-bound, iter: 9997 i: 5, pairs changed 5
non-bound, iter: 9997 i: 14, pairs changed 6
iteration number: 9998
non-bound, iter: 9998 i: 0, pairs changed 1
non-bound, iter: 9998 i: 1, pairs changed 2
non-bound, iter: 9998 i: 2, pairs changed 3
non-bound, iter: 9998 i: 3, pairs changed 4
non-bound, iter: 9998 i: 5, pairs changed 5
non-bound, iter: 9998 i: 14, pairs changed 6
iteration number: 9999
non-bound, iter: 9999 i: